In [1]:
!uv add youtube-transcript-api

Resolved 164 packages in 0.59ms
Audited 160 packages in 99ms


In [23]:
from youtube_transcript_api import YouTubeTranscriptApi
from openai import OpenAI
from pydantic import BaseModel
import pickle
import datetime

In [24]:
openai_client = OpenAI()

In [25]:
video_id = 'ph1PxZIkz1o'

In [26]:
#!wget https://github.com/alexeygrigorev/ai-bootcamp-codespace/raw/refs/heads/main/week1/ph1PxZIkz1o.bin

In [27]:
with open(f'{video_id}.bin', 'rb') as f_in:
    transcript = pickle.load(f_in)

In [30]:
print(transcript[:1])
type(transcript[:1])

[FetchedTranscriptSnippet(text='So hi everyone. Uh today we are going to', start=0.0, duration=5.04)]


list

In [8]:
def format_timestamp(seconds: float) -> str:
    """Convert seconds to H:MM:SS if > 1 hour, else M:SS"""
    total_seconds = int(seconds)
    hours, remainder = divmod(total_seconds, 3600)
    minutes, secs = divmod(remainder, 60)

    if hours > 0:
        return f"{hours}:{minutes:02}:{secs:02}"
    else:
        return f"{minutes}:{secs:02}"

In [9]:
def make_subtitles(transcript) -> str:
    lines = []

    for entry in transcript:
        ts = format_timestamp(entry.start)
        text = entry.text.replace('\n', ' ')
        lines.append(ts + ' ' + text)

    return '\n'.join(lines)


In [10]:
subtitles = make_subtitles(transcript)

In [11]:
print(subtitles[:500])

0:00 So hi everyone. Uh today we are going to
0:02 talk about our upcoming course. The
0:05 upcoming course is called machine
0:06 learning zoom camp. And um this is
0:10 already I put the link in the
0:12 description. So if you're watching um
0:14 this video in recording or you're
0:17 watching it live, you go here in the
0:19 description after under this video and
0:21 then you see a link course. uh click on
0:25 that link and this bring you will bring
0:27 you to
0:29 this website this GitHub


In [12]:
instructions = """
Summarize the transcript and describe the main purpose of the video
and the main ideas. 

Also output chapters with time. Use usual sentence case, not Title Case for the chapter.

Output format: 

<OUTPUT>
Summary

timestamp chapter 
timestamp chapter
...
timestamp chapter
</OUTPUT>
"""

In [13]:
def llm(user_prompt, instructions=None, model="gpt-4o-mini"):
    messages = []

    if instructions:
        messages.append({
            "role": "system",
            "content": instructions
        })

    messages.append({
        "role": "user",
        "content": user_prompt
    })

    response = openai_client.responses.create(
        model=model,
        input=messages
    )

    return response.output_text

In [14]:
answer = llm(subtitles, instructions=instructions)
print(answer)

<OUTPUT>
Summary

The video discusses an upcoming course titled "Machine Learning Zoom Camp," detailing its structure, content updates, prerequisites, and answering questions from potential participants. It highlights that the course will start on September 15th and will feature both updated and original content from previous editions. Participants are encouraged to ask questions through a pinned link. The course aims to teach machine learning engineering skills, with a focus on practical applications, deployment, and real-world projects. Although job placement isn't guaranteed, many past students found jobs after completing the course. 

Chapter timestamps:
0:00 introduction to the course
1:10 course overview
2:50 content updates
5:32 depth of computer vision and prerequisites
11:01 programming skills required
13:12 target audience: data scientists or ML engineers
16:30 certificates and milestones
22:50 time commitment and project expectations
30:27 course format and interaction
37:35

In [15]:
class Chapter(BaseModel):
    timestamp: str
    title: str

class YTSummaryResponse(BaseModel):
    summary: str
    chapters: list[Chapter]

In [16]:
instructions = """
Summarize the transcript and describe the main purpose of the video
and the main ideas. 

Also output chapters with time. Use usual sentence case, not Title Case for the chapter.

More chapters is better than fewer chapters. Have a chapter at least every 3-5 minutes
""".strip()

In [17]:
messages = [
    {"role": "system", "content": instructions}, 
    {"role": "user", "content": subtitles}
]

In [18]:
response = openai_client.responses.parse(
    model='gpt-4o-mini',
    input=messages,
    text_format=YTSummaryResponse
)

In [19]:
summary = response.output[0].content[0].parsed

print(summary.summary)
print()
for c in summary.chapters:
    print(c.timestamp, c.title)

In this video, the host outlines the details and updates for the upcoming "Machine Learning Zoom Camp" course. Key areas of focus include course structure, updates on content (notably transitioning from TensorFlow to PyTorch for some modules), prerequisites, and the intended audience. The goal of the course is to equip participants with practical skills for machine learning engineering jobs, especially regarding deployment and implementation rather than deep theoretical insights. The host also addresses various questions from potential students, clarifying expectations around job placement, prior knowledge needed, course materials, and project requirements.

0:00 Introduction to machine learning zoom camp
3:49 Course structure and content updates
6:47 Expectations and prerequisites for participants
14:49 Discussion on job placements and career outcomes
24:22 Course format and project details
31:04 Overview of practical commitments and time estimations
39:54 Final thoughts and upcoming 

In [20]:
def llm_structured(instructions, user_prompt, output_format, model="gpt-4o-mini"):
    messages = [
        {"role": "system", "content": instructions},
        {"role": "user", "content": user_prompt}
    ]

    response = openai_client.responses.parse(
        model=model,
        input=messages,
        text_format=output_format
    )

    return response.output[0].content[0].parsed

In [21]:
summary = llm_structured(
    instructions=instructions,
    user_prompt=subtitles,
    output_format=YTSummaryResponse
)

In [22]:
summary

YTSummaryResponse(summary='The video provides an overview of the upcoming "Machine Learning Zoom Camp" course. It includes details on course structure, modules, prerequisites, and answers to various queries about course content and objectives. The course is focused on machine learning engineering, covering fundamental concepts and practical applications, primarily focusing on Python, deployment, and minimal theory on computer vision and math.', chapters=[Chapter(timestamp='0:00', title='Introduction to the course'), Chapter(timestamp='5:20', title='Course content overview'), Chapter(timestamp='10:32', title='Practical programming prerequisites'), Chapter(timestamp='15:54', title='Deep learning and computer vision'), Chapter(timestamp='22:10', title='Course structure and project requirements'), Chapter(timestamp='30:34', title='Use of external resources for learning'), Chapter(timestamp='35:48', title='Expectations for job readiness after the course'), Chapter(timestamp='40:58', title='